In [31]:
import tensorflow as tf
import numpy as np
from sklearn import metrics

In [2]:
tf.reset_default_graph()

batch_size = 10

input_data = np.random.randint(0,10,[10,150,10]).astype(np.float32)
# print(input_data)

labels     = np.random.randint(0,10,[10,150])
print(labels)
sequence_length = np.random.randint(5,10,[10])




logits =       tf.placeholder(dtype=tf.float32,shape=[10,150,10])
labels_a =     tf.placeholder(dtype=tf.int32,shape=[10,150])
sequence_len =  tf.placeholder(dtype=tf.int32,shape=[10])


# logits :       A [batch_size, max_seq_len, num_tags] tensor of unary potentials to use as input to the CRF layer.

# labels_a :     A [batch_size, max_seq_len] matrix of tag indices for which we compute the log-likelihood.

# sequence_len : A [batch_size] vector of true sequence lengths.

[[2 4 8 ... 9 4 8]
 [3 8 4 ... 9 8 4]
 [7 1 8 ... 7 0 3]
 ...
 [5 0 4 ... 6 9 8]
 [8 2 1 ... 5 6 7]
 [2 1 4 ... 6 4 3]]


In [3]:
log_likelihood , transition_params = tf.contrib.crf.crf_log_likelihood(logits,labels_a,sequence_len)

#return of crf log_likelihood function

# log_likelihood: A scalar containing the log-likelihood of the given sequence of tag indices.
    
# transition_params: A [num_tags, num_tags] transition matrix. 

# This is either provided by the caller or created in this function.

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    log,transition = sess.run([log_likelihood , transition_params],feed_dict={logits:input_data,labels_a:labels,sequence_len:sequence_length})
    print(log)
    print('\n\n\n')
    print(transition)
    
    

[-29.617699  -56.336227  -30.197369  -51.90619   -20.026657  -12.888657
 -29.83575   -15.2441025 -43.371178  -24.594526 ]




[[ 0.13687605  0.02850056 -0.0191958  -0.3702659   0.441212    0.1264326
  -0.2744757  -0.1750347   0.23184067  0.1626041 ]
 [-0.37928763 -0.31222183  0.35007215 -0.05363819  0.3589471   0.01442468
  -0.22376254 -0.32380438 -0.04752463  0.40883964]
 [-0.07560167  0.09980613  0.08682263 -0.43068376  0.41416132  0.34156424
  -0.05819854  0.50512147  0.4461283   0.15736312]
 [ 0.37710643 -0.36421213  0.4110505  -0.30611306 -0.45531997 -0.38490433
  -0.24280855  0.39747906  0.15427774  0.10674369]
 [ 0.4090128   0.33476055 -0.37036932  0.17415404  0.16497833  0.10587674
   0.02286988 -0.06318411  0.29523408  0.26088208]
 [ 0.26114643  0.25977862 -0.3067152  -0.51419663  0.16423291 -0.27652317
   0.44483548 -0.46646205 -0.31327826  0.1184532 ]
 [ 0.37244797 -0.26589078 -0.53226614 -0.19419158  0.52410066  0.19643795
  -0.00187248  0.10245681  0.2852264   0.0822919 ]


In [4]:
#viterbi algorithm 
#since we padded our input with 0 
# but during veterbi algorithm we remove those padding 
# and takes the output till origional sequence length only

for i in range(batch_size):
    print("sequence_length {}   padded_logits {}   actual_length_logits {}".format(sequence_length[i],input_data[i].shape,input_data[i][:sequence_length[i]].shape))
    

sequence_length 7   padded_logits (150, 10)   actual_length_logits (7, 10)
sequence_length 9   padded_logits (150, 10)   actual_length_logits (9, 10)
sequence_length 5   padded_logits (150, 10)   actual_length_logits (5, 10)
sequence_length 8   padded_logits (150, 10)   actual_length_logits (8, 10)
sequence_length 8   padded_logits (150, 10)   actual_length_logits (8, 10)
sequence_length 5   padded_logits (150, 10)   actual_length_logits (5, 10)
sequence_length 6   padded_logits (150, 10)   actual_length_logits (6, 10)
sequence_length 5   padded_logits (150, 10)   actual_length_logits (5, 10)
sequence_length 8   padded_logits (150, 10)   actual_length_logits (8, 10)
sequence_length 5   padded_logits (150, 10)   actual_length_logits (5, 10)


In [83]:
def accuracy_calculation(real_,predicted_):
    

    score_ = []
    assert len(real_)==len(predicted_) , 'length should be same'
    
    for combined in zip(real_,predicted_):
        if combined[0]==combined[1]:
            score_.append(1)
        else:
            score_.append(0)
            
    return score_.count(1)/len(score_)



#Now_ we can calculate viterbi score 

# score: A [seq_len, num_tags] matrix of unary potentials.
# transition_params: A [num_tags, num_tags] matrix of binary potentials.

for seq_len, logits_data , label_ in zip(sequence_length,input_data,labels):
    
    score_ = np.array(logits_data[:seq_len]).astype(np.float32)
    transition_ = transition
    
#     print(score)           #seq_len x num_tags    
#     print(transition_params)#num_tags x num_tags
    
    viterbi,viterbi_score = tf.contrib.crf.viterbi_decode(score_,transition_)
    
    print( viterbi,viterbi_score,label_[:seq_len])
    print('\n')
    print("confidence_score",accuracy_calculation(viterbi,label_[:seq_len]))
    

    


[9, 9, 6, 6, 5, 3, 1] 58.67293 [2 4 8 6 5 4 9]


confidence_score 0.2857142857142857
[9, 4, 6, 0, 8, 6, 6, 1, 4] 78.94832 [3 8 4 6 0 8 1 0 1]


confidence_score 0.0
[3, 1, 7, 3, 2] 43.767082 [7 1 8 7 4]


confidence_score 0.2
[6, 5, 6, 2, 2, 2, 4, 1] 68.03157 [9 3 2 5 0 6 2 5]


confidence_score 0.0
[0, 9, 6, 6, 0, 4, 6, 4] 72.02991 [7 3 6 6 0 2 4 1]


confidence_score 0.375
[2, 4, 8, 3, 8] 45.48722 [1 3 3 4 7]


confidence_score 0.0
[0, 7, 5, 2, 6, 0] 48.104847 [9 3 1 1 0 9]


confidence_score 0.0
[8, 6, 4, 0, 1] 46.120716 [5 0 4 8 7]


confidence_score 0.2
[6, 4, 8, 0, 0, 9, 3, 3] 66.65506 [8 2 1 4 0 8 0 5]


confidence_score 0.125
[4, 6, 3, 2, 5] 44.581295 [2 1 4 0 2]


confidence_score 0.0
